In [1]:
# ──────────────────────────────────────────────────────────────────────────────
#  XY⇄YX  faithfulness  ·  APPLY  a previously-trained linear probe
# ──────────────────────────────────────────────────────────────────────────────
%cd ../..
%pwd
import os, re, json, time, random, logging
from pathlib import Path
from collections import defaultdict, Counter

import joblib
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix, classification_report,
    roc_curve, auc,
)

# ╭───────────────────────── CONFIGURABLE PARAMETERS ─────────────────────────╮
PROBE_PATH          = Path("linear_probes/realyesno_None4k/linear_probe_layer5.joblib")
ACTIVATIONS_DIR     = Path("h_hidden_space/outputs/f1_hint_xyyx/xyyx_deterministic/gt_lt_completions_1")
ANSWERS_DIRS        = [
    Path("e_confirm_xy_yx/outputs/matched_vals_gt"),
    Path("e_confirm_xy_yx/outputs/matched_vals_lt"),
]
QUESTION_JSON_ROOT  = Path("data/chainscope/questions_json/linked")

INFERENCE_BATCH_SIZE = 32        # must match the batch-size used when capturing hiddens
MAX_SAMPLES          = None      # e.g. 10_000 to subsample for speed, or None = use all
SEED                 = 0
# ╰────────────────────────────────────────────────────────────────────────────╯

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ─── logging setup ───────────────────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s  %(levelname)s │ %(message)s",
    datefmt="%H:%M:%S",
)
log = logging.getLogger("apply_probe")

t0 = time.time()
log.info("loading linear probe  →  %s", PROBE_PATH)
probe = joblib.load(PROBE_PATH)

m = re.search(r"layer(\d+)", PROBE_PATH.name)        # ← fixed: plain assignment
LAYER = int(m.group(1)) if m else None
if LAYER is None:
    raise RuntimeError("Could not infer layer-number from probe filename – "
                       "rename as “…layerXX.joblib” or set LAYER manually.")
log.info("   → expecting activations from transformer layer  %d", LAYER)

# ╭────────────────────────────────────────────────────────────────────────────╮
# │ 1.  answers-metadata lookup                                               │
# ╰────────────────────────────────────────────────────────────────────────────╯
answers_map = {}
for dir_ in ANSWERS_DIRS:
    for fp in dir_.glob("*.json"):
        with open(fp) as f:
            raw = json.load(f)
        questions = raw["questions"] if isinstance(raw, dict) else raw
        for q in questions:
            same_flag = q["same"][0] if isinstance(q["same"], list) else q["same"]
            q_no  = q["question_id"]
            q_yes = q["question_yes_id"]
            answers_map[q_no]  = {"expected": "NO",  "actual": q["a_answers"][0],
                                  "same": same_flag}
            answers_map[q_yes] = {"expected": "YES", "actual": q["b_answers"][0],
                                  "same": same_flag}

log.info("answers_map built with %d entries", len(answers_map))

# ╭────────────────────────────────────────────────────────────────────────────╮
# │ 2.  utilities to resolve hidden-file rows → question-ids                  │
# ╰────────────────────────────────────────────────────────────────────────────╯
_STEM2PATH = {}

def _index_datasets_once():
    if _STEM2PATH:               # already done
        return
    for p in QUESTION_JSON_ROOT.rglob("*.json"):
        _STEM2PATH[p.stem] = p
    if not _STEM2PATH:
        raise RuntimeError(f"No *.json found under {QUESTION_JSON_ROOT}")

def _dataset_questions_for(stem: str):
    _index_datasets_once()
    fp = _STEM2PATH.get(stem)
    if fp is None:
        raise FileNotFoundError(f"Cannot find dataset JSON for stem {stem}")
    with open(fp) as f:
        raw = json.load(f)
    return raw["questions"] if isinstance(raw, dict) else raw

_BATCH_RE = re.compile(r"_batch(\d+)_hidden\.pt$")

def question_ids_for_hidden_file(hid_path: Path, batch_size: int, actual_len: int):
    m = _BATCH_RE.search(hid_path.name)
    if m is None:
        raise ValueError(f"Bad hidden filename: {hid_path.name}")
    batch_idx = int(m.group(1))
    stem      = hid_path.name[:m.start()]         # strip “…_batchX_hidden.pt”
    q_list    = _dataset_questions_for(stem)
    start     = batch_idx * batch_size
    return [q_list[start + j]["question_id"] for j in range(actual_len)]

# ╭────────────────────────────────────────────────────────────────────────────╮
# │ 3.  collect hidden-vectors & labels                                       │
# ╰────────────────────────────────────────────────────────────────────────────╯
X_vecs, y_labels = [], []

hidden_files = sorted(ACTIVATIONS_DIR.rglob("*_hidden.pt"))
log.info("found %d hidden-state files in %s", len(hidden_files), ACTIVATIONS_DIR)

for hid_fp in hidden_files:
    batch_hidden = torch.load(hid_fp)            # list[n_layers]  each  (B,H)
    vecs_L       = batch_hidden[LAYER]           # (B,H) tensor for probed layer
    batch_len    = vecs_L.size(0)

    q_ids = question_ids_for_hidden_file(
        hid_fp, INFERENCE_BATCH_SIZE, batch_len
    )

    for row_idx, qid in enumerate(q_ids):
        meta = answers_map.get(qid)
        if meta is None:
            continue        # shouldn’t happen unless dirs mismatch

        correct = (meta["actual"] == meta["expected"])
        if not correct and not meta["same"]:
            continue        # we *skip* ordinary wrong answers

        label = 1 if correct else 0      # 1 = faithful-and-correct, 0 = same-answer error
        X_vecs.append(vecs_L[row_idx].float().numpy())
        y_labels.append(label)

log.info("collected %d labelled activation-vectors", len(X_vecs))
if not X_vecs:
    raise RuntimeError("Nothing to evaluate on – check the filters & paths.")

# optional subsample for speed
if MAX_SAMPLES and len(X_vecs) > MAX_SAMPLES:
    log.info("sub-sampling to %d items (random, reproducible)", MAX_SAMPLES)
    idx = np.random.RandomState(SEED).choice(len(X_vecs), MAX_SAMPLES, replace=False)
    X_vecs = [X_vecs[i] for i in idx]
    y_labels = [y_labels[i] for i in idx]

X = np.stack(X_vecs)
y = np.array(y_labels)

log.info("running probe on %s vectors (d=%d)", len(X), X.shape[1])

if hasattr(probe, "predict_proba"):
    prob_pos = probe.predict_proba(X)[:, 1]
else:
    prob_pos = probe.decision_function(X)
pred = (prob_pos >= 0.5).astype(int)

acc = accuracy_score(y, pred)
f1  = f1_score(y, pred)
log.info("RESULTS  ·  acc %.3f   f1 %.3f", acc, f1)
log.info("\n" + classification_report(y, pred, digits=3))

# ╭────────────────────────────────────────────────────────────────────────────╮
# │ 5.  quick-look figures                                                    │
# ╰────────────────────────────────────────────────────────────────────────────╯
figdir = PROBE_PATH.parent
figdir.mkdir(parents=True, exist_ok=True)

# Confusion-matrix
cm = confusion_matrix(y, pred)
plt.figure(figsize=(3,3))
plt.imshow(cm, cmap="Blues")
for (i,j),v in np.ndenumerate(cm):
    plt.text(j, i, f"{v:,}", ha="center", va="center", fontsize=9)
plt.xticks([0,1], ["error","correct"]); plt.yticks([0,1], ["error","correct"])
plt.title("confusion matrix")
plt.tight_layout(); cm_path = figdir / "confusion_matrix_apply.png"; plt.savefig(cm_path); plt.close()
log.info("• confusion-matrix written → %s", cm_path)

# ROC
fpr, tpr, _ = roc_curve(y, prob_pos)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, lw=2); plt.plot([0,1],[0,1],"--")
plt.title(f"ROC AUC {roc_auc:.3f}")
plt.xlabel("false-positive-rate"); plt.ylabel("true-positive-rate")
plt.tight_layout(); roc_path = figdir / "roc_apply.png"; plt.savefig(roc_path); plt.close()
log.info("• ROC curve written        → %s", roc_path)

# probability histogram
plt.figure(figsize=(5,3))
plt.hist(prob_pos[y==0], bins=40, alpha=0.6, label="label 0  (same-answer error)")
plt.hist(prob_pos[y==1], bins=40, alpha=0.6, label="label 1  (correct)")
plt.xlabel("probe p(correct)"); plt.ylabel("count"); plt.legend()
plt.tight_layout(); hist_path = figdir / "prob_hist_apply.png"; plt.savefig(hist_path); plt.close()
log.info("• probability histogram    → %s", hist_path)

log.info("done in %.1f s", time.time() - t0)


/root/CoTFaithChecker


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
16:07:14  INFO │ loading linear probe  →  linear_probes/realyesno_None4k/linear_probe_layer5.joblib
16:07:14  INFO │    → expecting activations from transformer layer  5
16:07:14  INFO │ answers_map built with 100 entries
16:07:14  INFO │ found 592 hidden-state files in h_hidden_space/outputs/f1_hint_xyyx/xyyx_deterministic/gt_lt_completions_1
/tmp/ipykernel_3871/2454956890.py:117: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future rele

In [3]:
# --- top-of-cell imports (replace the old block) -----------------------------
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    precision_recall_curve,
    average_precision_score,
    f1_score,
)
from sklearn.calibration import calibration_curve          # ← fixed line
from sklearn.decomposition import PCA
from pathlib import Path


figdir.mkdir(parents=True, exist_ok=True)   # already created earlier

# ── Precision–Recall curve ──────────────────────────────────────────────────
prec, rec, _ = precision_recall_curve(y, prob_pos)
ap = average_precision_score(y, prob_pos)
plt.figure(figsize=(4,4))
plt.plot(rec, prec, lw=2)
plt.xlabel("recall"); plt.ylabel("precision")
plt.title(f"PR  (AP = {ap:.3f})")
plt.grid(True)
pr_path = figdir / "pr_curve_apply.png"
plt.tight_layout(); plt.savefig(pr_path); plt.close()
log.info("• PR curve                → %s", pr_path)

# ── Calibration / reliability diagram ───────────────────────────────────────
true_frac, pred_prob = calibration_curve(y, prob_pos, n_bins=10, strategy="uniform")
plt.figure(figsize=(4,4))
plt.plot(pred_prob, true_frac, "o-", label="model")
plt.plot([0,1],[0,1],"--", label="perfect")
plt.xlabel("mean predicted prob"); plt.ylabel("fraction of positives")
plt.title("calibration")
plt.legend(); plt.grid(True)
cal_path = figdir / "calibration_apply.png"
plt.tight_layout(); plt.savefig(cal_path); plt.close()
log.info("• calibration curve       → %s", cal_path)

# ── Probability density by class ────────────────────────────────────────────
plt.figure(figsize=(5,3))
bins = np.linspace(0,1,50)
plt.hist(prob_pos[y==0], bins=bins, density=True, alpha=0.6,
         label="label 0  (same-answer error)")
plt.hist(prob_pos[y==1], bins=bins, density=True, alpha=0.6,
         label="label 1  (correct)")
plt.xlabel("probe p(correct)"); plt.ylabel("density"); plt.legend()
plt.title("probability density"); plt.grid(True, axis="y", alpha=.4)
dens_path = figdir / "prob_density_apply.png"
plt.tight_layout(); plt.savefig(dens_path); plt.close()
log.info("• probability density     → %s", dens_path)

# ── F1 vs threshold ─────────────────────────────────────────────────────────
thresholds = np.linspace(0,1,101)
f1s = [f1_score(y, prob_pos >= t) for t in thresholds]
plt.figure(figsize=(5,3))
plt.plot(thresholds, f1s)
plt.xlabel("threshold"); plt.ylabel("F1 score")
plt.title("F1 vs decision threshold"); plt.grid(True)
f1t_path = figdir / "f1_vs_threshold_apply.png"
plt.tight_layout(); plt.savefig(f1t_path); plt.close()
log.info("• F1-vs-threshold curve   → %s", f1t_path)

# ── 2-D PCA projection of layer activations ────────────────────────────────
try:
    pca = PCA(n_components=2, random_state=SEED)
    XY = pca.fit_transform(X)
    plt.figure(figsize=(4,4))
    plt.scatter(XY[y==0,0], XY[y==0,1], s=8, alpha=0.6, label="label 0")
    plt.scatter(XY[y==1,0], XY[y==1,1], s=8, alpha=0.6, label="label 1")
    plt.xlabel("PC 1"); plt.ylabel("PC 2"); plt.legend()
    plt.title("PCA of probed-layer activations")
    pca_path = figdir / "pca_apply.png"
    plt.tight_layout(); plt.savefig(pca_path); plt.close()
    log.info("• PCA scatter            → %s", pca_path)
except Exception as e:                                 # very high-dim? too few samples?
    log.warning("PCA plot skipped: %s", e)

log.info("all diagnostics written to %s", figdir)


16:07:52  INFO │ • PR curve                → linear_probes/realyesno_None4k/pr_curve_apply.png
16:07:52  INFO │ • calibration curve       → linear_probes/realyesno_None4k/calibration_apply.png
16:07:52  INFO │ • probability density     → linear_probes/realyesno_None4k/prob_density_apply.png
16:07:53  INFO │ • F1-vs-threshold curve   → linear_probes/realyesno_None4k/f1_vs_threshold_apply.png
16:07:59  INFO │ • PCA scatter            → linear_probes/realyesno_None4k/pca_apply.png
16:07:59  INFO │ all diagnostics written to linear_probes/realyesno_None4k
